In [1]:
import pandas as pd

# データフレームをロード
file_path = './exclusion.parquet'
df = pd.read_parquet(file_path)

# 使用する特徴量
features_list = ['color_histogram_std', 'orb_histogram_std', 'lbp_features_std', 'gabor_features_std', 'zernike_features_std']

# 除外候補列を追加
for feature in features_list:
    exclude_column = f"exclude_by_{feature}"
    df[exclude_column] = None  # 初期値をNoneに設定

# データフレームを保存
temp_file_path = './temp.parquet'
df.to_parquet(temp_file_path)
print(f"データフレームを保存しました: {temp_file_path}")

データフレームを保存しました: ./temp.parquet


In [775]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import networkx as nx

# データフレームをロード
file_path = './temp.parquet'
df = pd.read_parquet(file_path)

# 使用する特徴量
features_list = ['color_histogram_std', 'orb_histogram_std', 'lbp_features_std', 'gabor_features_std', 'zernike_features_std']

# 除外候補列を自動生成
for feature in features_list:
    exclude_column = f"exclude_by_{feature}"
    if exclude_column not in df.columns:  # 既存列がない場合に作成
        df[exclude_column] = None  # 初期値をNoneに設定

# 特徴量の選択
selected_feature = 'orb_histogram_std'
exclude_column = f"exclude_by_{selected_feature}"

# Noneが含まれる患者フォルダを列挙し、パス名でソート
pending_folders = np.sort(df[df[exclude_column].isnull()]['dirname'].unique())
print(f"処理対象の患者フォルダ数: {len(pending_folders)}")

# 処理対象が残っている場合のみ処理
if len(pending_folders) == 0:
    print("全ての患者フォルダの取捨選択が完了しています。")
else:
    # 処理対象のフォルダを自動的に選択
    selected_dirname = pending_folders[0]
    print(f"処理対象の患者フォルダ: {selected_dirname}")

    selected_images = df[df['dirname'] == selected_dirname]

    # 特徴量をテンソルに変換
    features = torch.tensor(np.stack(selected_images[selected_feature].values), dtype=torch.float32)

    # 特徴量を正規化してコサイン類似度を計算
    features_norm = features / features.norm(dim=1, keepdim=True)
    similarity_matrix = torch.mm(features_norm, features_norm.T).cpu().numpy()

    # 類似度が閾値を超えるペアを抽出
    threshold = 0.9  # 閾値
    high_similarity_pairs = np.argwhere(similarity_matrix > threshold)
    high_similarity_pairs = high_similarity_pairs[high_similarity_pairs[:, 0] < high_similarity_pairs[:, 1]]

    # グラフを構築
    G = nx.Graph()
    for idx1, idx2 in high_similarity_pairs:
        G.add_edge(idx1, idx2)

    clusters = list(nx.connected_components(G))
    print(f"抽出された連結成分数（クラスタ数）: {len(clusters)}")

    # 連結成分がない場合の処理
    if len(clusters) == 0:
        print(f"患者フォルダ '{selected_dirname}' には類似画像がありません。全ての画像を残します。")
        df.loc[selected_images.index, exclude_column] = False  # 全ての画像を残す
        temp_file_path = './temp.parquet'
        df.to_parquet(temp_file_path)
        print(f"患者フォルダ '{selected_dirname}' の処理結果を保存しました: {temp_file_path}")
    else:
        # 各クラスタごとに画像とチェックボックスを表示
        checkboxes = []
        rows = []

        for i, cluster in enumerate(clusters):
            cluster = sorted(cluster)  # クラスタ内のインデックスをソート
            print(f"クラスタ {i+1}: {len(cluster)} 枚の画像")

            # 横並びで画像を表示する
            images_per_row = 5

            for idx, img_idx in enumerate(cluster):
                
                img_path = selected_images['path'].iloc[img_idx]
                img = plt.imread(img_path)

                # メタデータを取得
                width = selected_images['width'].iloc[img_idx]
                height = selected_images['height'].iloc[img_idx]
                created_time = pd.to_datetime(selected_images['created_time'].iloc[img_idx]).strftime('%Y-%m-%d %H:%M')
                modified_time = pd.to_datetime(selected_images['modified_time'].iloc[img_idx]).strftime('%Y-%m-%d %H:%M')

                # 画像表示用のウィジェット
                image_widget = widgets.Output()
                with image_widget:
                    plt.figure(figsize=(6, 6))
                    plt.imshow(img)
                    plt.axis('off')
                    plt.title(f"{width}x{height}\n{created_time}\n{modified_time}", fontsize=10)  # タイトルのフォントサイズも調整
                    plt.show()

                # チェックボックスを作成
                checkbox = widgets.Checkbox(value=False, description=f"Keep {img_idx}")
                checkboxes.append((img_idx, checkbox))

                # 一行分の画像とチェックボックスをまとめる
                if idx % images_per_row == 0:
                    rows.append([])  # 新しい行を開始
                rows[-1].append(widgets.VBox([image_widget, checkbox]))

        # 各行を水平方向に配置
        cluster_ui = widgets.VBox([widgets.HBox(row) for row in rows])

        # 表示
        display(cluster_ui)

        # 出力ウィジェット
        output = widgets.Output()

        # 取捨選択ボタン
        button = widgets.Button(description="Submit Selection")

        def on_button_click(b):
            with output:
                # チェックボックスで選択された画像を保存
                for img_idx, checkbox in checkboxes:
                    df.loc[selected_images.index[img_idx], exclude_column] = not checkbox.value  # チェックされていないものをTrueに
                
                # 類似画像がないものを自動的に残す
                no_similarity_indices = selected_images.index.difference(
                    df[df[exclude_column].notnull()].index
                )
                df.loc[no_similarity_indices, exclude_column] = False

                # データフレームを保存
                temp_file_path = './temp.parquet'
                df.to_parquet(temp_file_path)
                
                # 保存成功メッセージを表示
                print(f"患者フォルダ '{selected_dirname}' の処理結果を保存しました: {temp_file_path}")

        button.on_click(on_button_click)
        display(button, output)


処理対象の患者フォルダ数: 0
全ての患者フォルダの取捨選択が完了しています。


In [75]:
# orb_histogram_std で類似度検索

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import networkx as nx

# データフレームをロード
file_path = './temp.parquet'
df = pd.read_parquet(file_path)

# 使用する特徴量
features_list = ['color_histogram_std', 'orb_histogram_std', 'lbp_features_std', 'gabor_features_std', 'zernike_features_std']

# 除外候補列を自動生成
for feature in features_list:
    exclude_column = f"exclude_by_{feature}"
    if exclude_column not in df.columns:  # 既存列がない場合に作成
        df[exclude_column] = None  # 初期値をNoneに設定

# 特徴量の選択
selected_feature = 'orb_histogram_std'
exclude_column = f"exclude_by_{selected_feature}"

# 前のフィルタリング結果を利用
# `exclude_by_color_histogram_std` 列で False になっている行だけを対象にする
previous_exclude_column = 'exclude_by_color_histogram_std'
filtered_df = df[df[previous_exclude_column] == False]

# Noneが含まれる患者フォルダを列挙し、パス名でソート
pending_folders = np.sort(filtered_df[filtered_df[exclude_column].isnull()]['dirname'].unique())
print(f"処理対象の患者フォルダ数: {len(pending_folders)}")

# 処理対象が残っている場合のみ処理
if len(pending_folders) == 0:
    print("全ての患者フォルダの取捨選択が完了しています。")
else:
    # 処理対象のフォルダを自動的に選択
    selected_dirname = pending_folders[0]
    print(f"処理対象の患者フォルダ: {selected_dirname}")

    selected_images = filtered_df[filtered_df['dirname'] == selected_dirname]

    # 特徴量をテンソルに変換
    features = torch.tensor(np.stack(selected_images[selected_feature].values), dtype=torch.float32)

    # 特徴量を正規化してコサイン類似度を計算
    features_norm = features / features.norm(dim=1, keepdim=True)
    similarity_matrix = torch.mm(features_norm, features_norm.T).cpu().numpy()

    # 類似度が閾値を超えるペアを抽出
    threshold = 0.94  # 閾値
    high_similarity_pairs = np.argwhere(similarity_matrix > threshold)
    high_similarity_pairs = high_similarity_pairs[high_similarity_pairs[:, 0] < high_similarity_pairs[:, 1]]

    # グラフを構築
    G = nx.Graph()
    for idx1, idx2 in high_similarity_pairs:
        G.add_edge(idx1, idx2)

    clusters = list(nx.connected_components(G))
    print(f"抽出された連結成分数（クラスタ数）: {len(clusters)}")

    # 連結成分がない場合の処理
    if len(clusters) == 0:
        print(f"患者フォルダ '{selected_dirname}' には類似画像がありません。全ての画像を残します。")
        df.loc[selected_images.index, exclude_column] = False  # 全ての画像を残す
        temp_file_path = './temp.parquet'
        df.to_parquet(temp_file_path)
        print(f"患者フォルダ '{selected_dirname}' の処理結果を保存しました: {temp_file_path}")
    else:
        # 各クラスタごとに画像とチェックボックスを表示
        checkboxes = []
        rows = []

        for i, cluster in enumerate(clusters):
            cluster = sorted(cluster)  # クラスタ内のインデックスをソート
            print(f"クラスタ {i+1}: {len(cluster)} 枚の画像")

            # 横並びで画像を表示する
            images_per_row = 5

            for idx, img_idx in enumerate(cluster):
                img_path = selected_images['path'].iloc[img_idx]
                img = plt.imread(img_path)

                # メタデータを取得
                width = selected_images['width'].iloc[img_idx]
                height = selected_images['height'].iloc[img_idx]
                created_time = pd.to_datetime(selected_images['created_time'].iloc[img_idx]).strftime('%Y-%m-%d %H:%M')
                modified_time = pd.to_datetime(selected_images['modified_time'].iloc[img_idx]).strftime('%Y-%m-%d %H:%M')

                # 画像表示用のウィジェット
                image_widget = widgets.Output()
                with image_widget:
                    plt.figure(figsize=(6, 6))
                    plt.imshow(img)
                    plt.axis('off')
                    plt.title(f"{width}x{height}\n{created_time}\n{modified_time}", fontsize=10)  # タイトルのフォントサイズも調整
                    plt.show()

                # チェックボックスを作成
                checkbox = widgets.Checkbox(value=False, description=f"Keep {img_idx}")
                checkboxes.append((img_idx, checkbox))

                # 一行分の画像とチェックボックスをまとめる
                if idx % images_per_row == 0:
                    rows.append([])  # 新しい行を開始
                rows[-1].append(widgets.VBox([image_widget, checkbox]))

        # 各行を水平方向に配置
        cluster_ui = widgets.VBox([widgets.HBox(row) for row in rows])

        # 表示
        display(cluster_ui)

        # 出力ウィジェット
        output = widgets.Output()

        # 取捨選択ボタン
        button = widgets.Button(description="Submit Selection")

        def on_button_click(b):
            with output:
                # チェックボックスで選択された画像を保存
                for img_idx, checkbox in checkboxes:
                    df.loc[selected_images.index[img_idx], exclude_column] = not checkbox.value  # チェックされていないものをTrueに
                
                # 類似画像がないものを自動的に残す
                no_similarity_indices = selected_images.index.difference(
                    df[df[exclude_column].notnull()].index
                )
                df.loc[no_similarity_indices, exclude_column] = False

                # データフレームを保存
                temp_file_path = './temp.parquet'
                df.to_parquet(temp_file_path)
                
                # 保存成功メッセージを表示
                print(f"患者フォルダ '{selected_dirname}' の処理結果を保存しました: {temp_file_path}")

        button.on_click(on_button_click)
        display(button, output)


処理対象の患者フォルダ数: 0
全ての患者フォルダの取捨選択が完了しています。


In [91]:
# lbp_features_std で類似度検索

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import networkx as nx

# データフレームをロード
file_path = './temp.parquet'
df = pd.read_parquet(file_path)

# 使用する特徴量
features_list = ['color_histogram_std', 'orb_histogram_std', 'lbp_features_std', 'gabor_features_std', 'zernike_features_std']

# 除外候補列を自動生成
for feature in features_list:
    exclude_column = f"exclude_by_{feature}"
    if exclude_column not in df.columns:  # 既存列がない場合に作成
        df[exclude_column] = None  # 初期値をNoneに設定

# 特徴量の選択
selected_feature = 'lbp_features_std'
exclude_column = f"exclude_by_{selected_feature}"

# すでに除外作業の終わった列をリストとして管理
previous_exclude_columns = ['exclude_by_color_histogram_std', 'exclude_by_orb_histogram_std']

# 前のフィルタリング結果を利用
# すべての previous_exclude_columns で False になっている行だけを対象にする
filtered_df = df[
    df[previous_exclude_columns].apply(lambda row: all(row == False), axis=1)
]

# Noneが含まれる患者フォルダを列挙し、パス名でソート
pending_folders = np.sort(filtered_df[filtered_df[exclude_column].isnull()]['dirname'].unique())
print(f"処理対象の患者フォルダ数: {len(pending_folders)}")

# 処理対象が残っている場合のみ処理
if len(pending_folders) == 0:
    print("全ての患者フォルダの取捨選択が完了しています。")
else:
    # 処理対象のフォルダを自動的に選択
    selected_dirname = pending_folders[0]
    print(f"処理対象の患者フォルダ: {selected_dirname}")

    selected_images = filtered_df[filtered_df['dirname'] == selected_dirname]

    # 特徴量をテンソルに変換
    features = torch.tensor(np.stack(selected_images[selected_feature].values), dtype=torch.float32)

    # 特徴量を正規化してコサイン類似度を計算
    features_norm = features / features.norm(dim=1, keepdim=True)
    similarity_matrix = torch.mm(features_norm, features_norm.T).cpu().numpy()

    # 類似度が閾値を超えるペアを抽出
    threshold = 0.92  # 閾値
    high_similarity_pairs = np.argwhere(similarity_matrix > threshold)
    high_similarity_pairs = high_similarity_pairs[high_similarity_pairs[:, 0] < high_similarity_pairs[:, 1]]

    # グラフを構築
    G = nx.Graph()
    for idx1, idx2 in high_similarity_pairs:
        G.add_edge(idx1, idx2)

    clusters = list(nx.connected_components(G))
    print(f"抽出された連結成分数（クラスタ数）: {len(clusters)}")

    # 連結成分がない場合の処理
    if len(clusters) == 0:
        print(f"患者フォルダ '{selected_dirname}' には類似画像がありません。全ての画像を残します。")
        df.loc[selected_images.index, exclude_column] = False  # 全ての画像を残す
        temp_file_path = './temp.parquet'
        df.to_parquet(temp_file_path)
        print(f"患者フォルダ '{selected_dirname}' の処理結果を保存しました: {temp_file_path}")
    else:
        # 各クラスタごとに画像とチェックボックスを表示
        checkboxes = []
        rows = []

        for i, cluster in enumerate(clusters):
            cluster = sorted(cluster)  # クラスタ内のインデックスをソート
            print(f"クラスタ {i+1}: {len(cluster)} 枚の画像")

            # 横並びで画像を表示する
            images_per_row = 5

            for idx, img_idx in enumerate(cluster):
                img_path = selected_images['path'].iloc[img_idx]
                img = plt.imread(img_path)

                # メタデータを取得
                width = selected_images['width'].iloc[img_idx]
                height = selected_images['height'].iloc[img_idx]
                created_time = pd.to_datetime(selected_images['created_time'].iloc[img_idx]).strftime('%Y-%m-%d %H:%M')
                modified_time = pd.to_datetime(selected_images['modified_time'].iloc[img_idx]).strftime('%Y-%m-%d %H:%M')

                # 画像表示用のウィジェット
                image_widget = widgets.Output()
                with image_widget:
                    plt.figure(figsize=(6, 6))
                    plt.imshow(img)
                    plt.axis('off')
                    plt.title(f"{width}x{height}\n{created_time}\n{modified_time}", fontsize=10)  # タイトルのフォントサイズも調整
                    plt.show()

                # チェックボックスを作成
                checkbox = widgets.Checkbox(value=False, description=f"Keep {img_idx}")
                checkboxes.append((img_idx, checkbox))

                # 一行分の画像とチェックボックスをまとめる
                if idx % images_per_row == 0:
                    rows.append([])  # 新しい行を開始
                rows[-1].append(widgets.VBox([image_widget, checkbox]))

        # 各行を水平方向に配置
        cluster_ui = widgets.VBox([widgets.HBox(row) for row in rows])

        # 表示
        display(cluster_ui)

        # 出力ウィジェット
        output = widgets.Output()

        # 取捨選択ボタン
        button = widgets.Button(description="Submit Selection")

        def on_button_click(b):
            with output:
                # チェックボックスで選択された画像を保存
                for img_idx, checkbox in checkboxes:
                    df.loc[selected_images.index[img_idx], exclude_column] = not checkbox.value  # チェックされていないものをTrueに
                
                # 類似画像がないものを自動的に残す
                no_similarity_indices = selected_images.index.difference(
                    df[df[exclude_column].notnull()].index
                )
                df.loc[no_similarity_indices, exclude_column] = False

                # データフレームを保存
                temp_file_path = './temp.parquet'
                df.to_parquet(temp_file_path)
                
                # 保存成功メッセージを表示
                print(f"患者フォルダ '{selected_dirname}' の処理結果を保存しました: {temp_file_path}")

        button.on_click(on_button_click)
        display(button, output)

処理対象の患者フォルダ数: 762
処理対象の患者フォルダ: 21
抽出された連結成分数（クラスタ数）: 0
患者フォルダ '21' には類似画像がありません。全ての画像を残します。
患者フォルダ '21' の処理結果を保存しました: ./temp.parquet
